![Quantinuum_logo](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIIAAABLCAMAAACLOq%2BzAAACx1BMVEUAAAAAAP%2BAgIBVqqqAgIBmmZmAgKptkpKAgJ9xjo6AgJl0i6JqgJV2iZ1tgJJ3iJlwgJ94h5ZxgJx5hpRzgJl5hp50gJdvhZt1gJVwhZl2gJ12gJtyhJV3gJlzhJx4gJd0g5txgJZ1g5lxgJx1g5hygJp2g5ZzgJl2g5tzgJh3gpp0gJdxgpl0gJtygph1gJpygpd1gJlzgpt2gJhzgpp2gJd0gplygJt0gphygJp1gpdzgJl1gptzgJh1gpp2gZl0gJtygZh0gJpzgZh1gJlzgZp1gJhzgZp1gJh0gZl0gZhygJp0gZhzgJl0gZpzgJh1gZpzgJh1gZl0gJp1gZh0gJp0gJlzgZp0gJhzgZp1gJhzgZl1gJp0gZh1gJp0gZhzgJl0gZpzgJlzgJh1gZlzgJp1gZl1gZh0gJl0gJlzgZl0gJhzgZl1gJp0gZl1gJl0gZh1gJl0gZpzgJl0gZlzgJh0gZlzgJp0gZl0gJl1gJh1gJpzgJl0gJhzgJl0gJpzgJl0gJl1gJl0gJp1gJl0gJl1gJhzgJp0gJlzgJl0gJh0gJl1gJp0gJl1gJl0gJh1gJl0gJpzgJl0gJlzgJh0gJl0gJp0gJl0gJl1gJh0gJl1gJp0gJlzgJl0gJh0gJl0gJp0gJl0gJl0gJh1gJl0gJpzgJh0gJl0gJl0gJl1gJp0gJl1gJl0gJhzgJl0gJl0gJl0gJh0gJl0gJp0gJl0gJl1gJh0gJlzgJp0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJlzgJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJlzgJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJn%2F%2F%2F%2Bc0JCdAAAA63RSTlMAAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYGRocHR4fICEiIyQlJicoKSorLC0uLzAxMjM0NTY3ODk6Ozw9Pj9BQkNERUZHSElKS01OT1BRUlNUVVZXWFpbXF1eX2BhYmNkZWZoaWprbW5wcXJzdHV2d3h5ent8fX5%2FgIGDhYaHiImKjI2Oj5CSk5SVlpeYmZqbnJ2en6ChoqOkpaanqKmqq6ytrq%2Bws7a3ubq7vL2%2BwMHCw8TFxsfIycrLzM3Oz9DR0tPU1dbY2drb3N3e3%2BDh4uPk5ebn6Orr7O3u7%2FHy8%2FT19vf4%2Bfr7%2FP3%2B7Jw5zgAAAAFiS0dE7CG5sxsAAAVfSURBVBgZ7cGNV9X1AcfxNxcwnhRRaVCsKGszI%2BbDcs3KlDKldAmhGywz0ZyJOqLUHrBraVoKJD5E%2BBDaarXS2tZ0jmkqKjlLKR9SA6EFCNzPP7Hv7%2F7QmMm9nM754jkdXi9%2BsNCxL7x9uLb266qKZx%2BI5Ar4yYsn9J2GNyd46F6e%2BfW6RFWmh2404ANdxq6f022uOajL%2BvYPIXSPyN3qTFkY1vUBytW5tyOwLHE%2BjFMg73iwKqpyCCEHFFAhVpXsgfEKzJeBRWO1CDYpiK%2FisCbsU91N%2BDkFU4Q1WVIkqWrX%2BvGTv04Ih96Dp246o47aUrBlpxogR351z8XzHc%2F4j9TBGiz5lVQD82W0Lo7lErfv0EWN8dixTDoICyTV3Mn3heY36YK52LFf%2BgLypH%2FHcVlDT6vddqxI8EnfQLb29qMTg0%2FJ1RSNDQ%2FJiGJEVTydGlQv1%2F3YMEdGGp4BBJAt1xPYsFTGYoLYIL8l2LBZRhVBJDXJ8TI2%2FEmOewhilRwrsGGDHO8QxI1yvIgNa%2BQ3hiCqZCzChmXyq76KwBbLmIsN0%2BQqIbAHZEzGhmFql0dAKTKSsCGiRa622QQSI6kGO7bpgqWhBNAgrcSOabroX7fRuQ8%2Fqx6JHX2bdNH55ddxJZSrg5aNE6Lpdre06v80bnvVW5DfbnZuR6OxZKW6qhxL%2Btaoi97Altub1TWvY81jPnXJeuzJV5esw6LHW9UFa7Ep%2FayCK8Wqaz9QUKux7OFjCqIE28If2aeAiukGtxX%2B47w6VUT3iBg8Mdc1K%2F8S99GjR48ePXr06PHjkzRjycIxHhyR0%2BNwxOT0wxH64G%2F74xrzaDR%2B9%2F4SeDDzghDwZCZh3PdoJH7jhgLXZobhiH84Aph4E36TBgGpmem0S52eDuFLz8vYPwRjiMbj%2BJlycGRLr%2BKq1N964zi0HqjXBTEQockYB7UtGsexImCSYnHMVCJQl4cjwrcAWCaNxy%2F2tE4SskXb74i%2B5omGhhRgpDJwpGgGRtjhszuarsNvd5t2xmJ8XgYMHZ2WdqQuLS1tWBhEaQrGoTb9tTfGVyXAJF2NI0%2BJQF0Bjr5aACz31X%2FaC8cSfXOWDK0LwfjFt7tC4E49gmOEpmFMVW5KSyl%2Be%2F68QJX9gFPluHafwBWj32FUbynUjljg3GrgIf0UR74SgbrncAzQ08CKs7M0DyO56d2t9Ww%2FFYPfUxoOqZqN435lAeGf7fVQ0jIQx97NzNG%2BBKjbiKvyGK4%2BysE4vJYC7Y6H5lJgggbheF6JwJmXcFyvfOCVk6H7ziUAFa23ljdzeg2uFM2CG7QQxxSNA6ZrEiS3lOPYvwHm6EgyjZtx7TqCq69%2Bj3FkNTyp6iSP1gLpugvHMiUCp9bjSFEesPILRqkM7tFrlLbS8Bqu6%2FVHCP36n6EYmxv7QK9jenxKVu6J1sEYB8qAuTpxq68C18fVuOI0FePzYqBANalaDyRrHUbvmuO9gA%2FPxGE8rVFA0VHY4rsr7EBDAkUtHNiJK10ZwHy9lUD0cnmBmWq3FeNQGUaBarUV1%2FYqXP01DeNoMcYi1aoMY5MKI0neoRyMe31%2Fv5FeeS0fhQAlR2Fg8755WgirmnlWo%2FCLHRgKeFZIdY16Ixyu%2BrJy%2BDBHqW8IUF2G4xnpLVzvfYIrXtMxaopxPC%2BVY8T8ReePy1eI38xmX81%2FtedqjNVHgZfV9mUUrGok%2FvjpsXQ0svj90jSMG7zD8Yt%2BZiwwLwu%2FXG82rqzHcMV4R2AU%2FAa%2FGd7JOEImvv7e8mG0u3nJuxuzw3Fk5AP9vd7RwMQCGPQfHayoKClZ6R3FlRJV8MnJ2trG2trZXBH%2FA2RBOeTINIkQAAAAAElFTkSuQmCC)<br>
# Mid-Circuit Measurement

This notebook contains an example using mid-circuit measurement using the Quantinuum machines.

## Repetition Code Circuit

The use of mid-circuit measurement is straightforward, note the use of `measure` and `reset` on the ancilla qubits. This example also utlizes conditional logic available with Quantinuum devices as well as Registers and IDs available in `pytket`. See [Conditional Gates](https://cqcl.github.io/pytket/manual/manual_circuit.html#classical-and-conditional-operations) and [Registers and IDs](https://cqcl.github.io/pytket/manual/manual_circuit.html#registers-and-ids) for additional examples.

In [ ]:
from pytket.circuit import Circuit, Qubit, Bit, OpType, reg_eq
from pytket.circuit.display import render_circuit_jupyter

Set up Repetition Code with mid-circuit measurement and corrections

2    1    0 = data: data qubits<br>
*----*----*<br>
  ZZ   ZZ<br>
  1    0    = syndromes<br>
  0    0    = ancillas

Set up circuit object

In [ ]:
circuit = Circuit(name="Repetition Code")

Reserve registries

Add qubit register, the data qubits

In [ ]:
data = circuit.add_q_register("data", 3)

Add qubit register, the ancilla qubit

In [ ]:
ancilla = circuit.add_q_register("anc", 1)

Add classical registers for the syndromes

In [ ]:
syndrome = circuit.add_c_register("syndrome", 2)

Add classical registers for the output

In [ ]:
output = circuit.add_c_register("output", 3)

Prepare the logical state<br>
Qubits always start in |0> and logical |0> == |000>.<br>
So we already start in logical |0>.

Syndrome Extraction

In [ ]:
circuit.add_gate(OpType.Reset, ancilla)
circuit.CX(data[0], ancilla[0])
circuit.CX(data[1], ancilla[0])
circuit.Measure(ancilla[0], syndrome[0])

Syndrome Extraction

In [ ]:
circuit.add_gate(OpType.Reset, ancilla)
circuit.CX(data[1], ancilla[0])
circuit.CX(data[2], ancilla[0])
circuit.Measure(ancilla[0], syndrome[1])

Correction<br>
# if(syndromes==1) -> 01 -> check 0 bad -> X on qubit 0

In [ ]:
circuit.X(data[0], condition=reg_eq(syndrome, 1))

# if(syndromes==2) -> 10 -> check 1 bad -> X on qubit 2

In [ ]:
circuit.X(data[2], condition=reg_eq(syndrome, 2))

# if(syndromes==3) -> 11 -> check 1 and 2 bad -> X on qubit 1

In [ ]:
circuit.X(data[1], condition=reg_eq(syndrome, 3))

Measure out data qubits

In [ ]:
circuit.Measure(data[0], output[0])
circuit.Measure(data[1], output[1])
circuit.Measure(data[2], output[2])

In [ ]:
render_circuit_jupyter(circuit)

## Select Device

Login to the Quantinuum API using your credentials and check the device status.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

In [ ]:
machine = "H1-1E"

In [ ]:
backend = QuantinuumBackend(device_name=machine)

In [ ]:
backend.login()

In [ ]:
print(machine, "status:", backend.device_state(device_name=machine))

### Circuit Compilation

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.

Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimisation to perform during compilation, check pytket documentation for more details.

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)

In [ ]:
render_circuit_jupyter(compiled_circuit)

## Submit and Run the Circuit

In [ ]:
n_shots = 100
print(
    "Cost in HQCs:",
    backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker="H1-1SC"),
)

In [ ]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

In [ ]:
status = backend.circuit_status(handle)
print(status)

In [ ]:
import json

In [ ]:
result = backend.get_result(handle)

In [ ]:
with open("pytket_mcmr_example.json", "w") as file:
    json.dump(result.to_dict(), file)

## Analyze Results

We will now take the raw results and apply a majority vote to determine how many times we got 0 vs 1.

First, define a majority vote function.

In [ ]:
def majority(result):
    """Returns whether the output should be considered a 0 or 1."""
    if result.count(0) > result.count(1):
        return 0
    elif result.count(0) < result.count(1):
        return 1
    else:
        raise Exception("count(0) should not equal count(1)")

Now process the output:

In [ ]:
result_output_cnts = result.get_counts([output[i] for i in range(output.size)])

In [ ]:
result_output_cnts

In [ ]:
zeros = 0  # Counts the shots with majority zeros
ones = 0  # Counts the shots with majority ones

In [ ]:
for out in result_output_cnts:
    m = majority(out)
    if m == 0:
        zeros += result_output_cnts[out]
    else:
        ones += result_output_cnts[out]

A logical zero was initialized, so our error rate should be number of ones / total number of shots: `ones/shots`

In [ ]:
p = ones / n_shots
print(f"The error-rate is: p = {p}")

<div align="center"> &copy; 2023 by Quantinuum. All Rights Reserved. </div>